파이썬 머신러닝 완벽가이드 9장 확인 필요

In [4]:
import numpy as np
from sklearn.metrics import mean_squared_error

# 예제 데이터 생성

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))


def get_rmse(R, P, Q, non_zeros) :
    error = 0
    
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 Null이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse   

# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j , R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j ] > 0]  

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps) :
    for i, j, r in non_zeros : 
        
        # 실제 값과 예측 값의 차이인 오류값 구하기
        eij = r - np.dot(P[i, :], Q[j,:].T)
        
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
        Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])


    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 100) == 0 :
        print('### iteration step:', step, ' rmse: ', rmse)       
        

### iteration step: 0  rmse:  3.2388050277987723
### iteration step: 100  rmse:  0.1564340384819247
### iteration step: 200  rmse:  0.04325226798579314
### iteration step: 300  rmse:  0.022621116143829466
### iteration step: 400  rmse:  0.018022719092132704
### iteration step: 500  rmse:  0.016973657887570753
### iteration step: 600  rmse:  0.01670132290188466
### iteration step: 700  rmse:  0.016605910068210026
### iteration step: 800  rmse:  0.01654431582921597
### iteration step: 900  rmse:  0.01648146573819501


In [5]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [6]:
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])